# Fusion de tables

La fusion de tables consiste à __réunir en une seule table des données__ présentes dans plusieurs tables.

Il existe deux grands types de fusion de tables :

- __La réunion de tables__ : lorsque les descripteurs des deux tables sont identiques (par exemple dans le cadre de l'obtention de données plus récentes), on peut facilement réunir l'ensemble des données en une seule table.
- __La jointure de tables__ : lorsque deux tables ont au moins un descripteur en commun, il est également possible de fusionner ces tables pour créer une nouvelle table comprenant ce descripteur commun et d'autres, au choix.


## Pourquoi fusionner des tables ?

Reprenons notre table des naissances nantaises...

Admettons que nous récupérions un fichier CSV avec __uniquement les naissances nantaises de l'année 2020__, il serait intéressant d'intégrer ces nouvelles données à notre précédente table pour obtenir une __table de données "mise à jour"__. Dans ce cas, il faudra procédéer à une __réunion de table__ car on peut considérer comme acquis que __tous les descripteurs soient conservés d'une année sur l'autre__.

Considérons maintenant que nous trouvions un autre fichier, ayant pour descripteurs :

- des prénoms
- leur étymologie
- leur histoire
- des personnage célèbres ayant porté ce prénom
- etc

Si on veut créer un site web sur les prénoms, il serait très intéressant d'avoir une table réunissant, à la fois ces dernières données, mais aussi les occurrences de naissance. Dans ce cas, __seul le descripteur "prénom" est commun aux deux tables, il faudra donc procéder à une jointure__.

## Comment fusionner deux tables ?

### Réunion de tables
#### Mise à jour d'une table avec des données récentes

Vous avez à votre disposition un __fichier avec des données fictives__, crée uniquement pour cette activité NSI : 2020_fictif_prenoms-enfants-nes-nantes.csv

Le code suivant crée __deux tables, à partir des deux fichiers__ à notre disposition :

In [ ]:
import csv

with open("244400404_prenoms-enfants-nes-nantes.csv", mode='r', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter=';')
    table_naissances = [{key : value for key, value in element.items()} for element in reader]

with open("2020_fictif_prenoms-enfants-nes-nantes.csv", mode='r', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter=';')
    table_naissances_2020 = [{key : value for key, value in element.items()} for element in reader]

print(table_naissances_2020)

L'objectif est de __réunir les données de naissances 2020 à la table créee à partir du fichier de l'open data nantais__.

Il faut donc simplement parcourir la table 2020 et ajouter chacun des éléments qu'elle contient à la table principale :

In [ ]:
for bebe in table_naissances_2020:
    table_naissances.append(bebe)

print(table_naissances)

Nous voici donc avec une __nouvelle table, mise à jour__ avec toutes les naissances nantaises de 2001 à 2020. 

### Jointure de tables

> __Remarques :__
- l'exemple choisi dans la suite de ce notebook incite à effectuer une jointure sur index plutôt que sur tables. Le principe est toufois le même mais, __si vous préférez étudier une vraie jointure de tables, plus simple, passez directement au prochain notebook, NSI_P4, appliquant une jointure pour le projet "Choixpeau magique"__.
- la suite de ce notebook n'étant pas simple, les activités y sont directement corrigées. Toutefois, __si vous vous sentez en confiance__, il vous est toujours possible d'effacer le contenu de certaines cellules de code pour faire vous même les activités.

Afin de __comparer les naissances nantaises et les naissances rennaises__, nous allons créer une table réunissant les données de ces deux villes. On parle de __jointure de table car elles ont au moins un descripteur en commun : "Prénom"__.

Nous allons __créer une table permettant de savoir si tel ou tel prénom est davantage donné à Nantes ou à Rennes, année par année__.

Nous pouvons reprendre la table nantaise, sous sa forme indexée par prénom.

> __Remarque :__ on en profite au passage pour transformer le type des occurrences en entier, ce qui facilitera les comparaisons ultérieures.

In [ ]:
import csv

with open("244400404_prenoms-enfants-nes-nantes.csv", mode='r', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter=';')
    naissances_nantaises = [{key : value for key, value in element.items()} for element in reader]

index_nantais = {}

for element in naissances_nantaises:
    if element['Prénom'] in index_nantais:
        index_nantais[element['Prénom']].update({element['Année de naissance'] : int(float(element['Occurence']))})
    else:
        index_nantais[element['Prénom']] = {element['Année de naissance'] : int(float(element['Occurence']))}

print(index_nantais)

Reste à __faire de même avec les données rennaises__, téléchargées à partir du [site d'open data de la Métropole de Rennes](https://data.rennesmetropole.fr/explore/dataset/prenoms-a-rennes/).

Nous ne reprendrons que les attributs qui nous intéressent : ceux concernant les prénoms et les occurrences de naissances pour chaque année.

In [ ]:
with open("prenoms-a-rennes.csv", mode='r', encoding='utf-8') as f:
    reader = csv.DictReader(f, delimiter=';')
    naissances_rennaises = [{key : value for key, value in element.items()} for element in reader]

index_rennais = {}

for element in naissances_rennaises:
    if element['Prénom'] in index_rennais:
        index_rennais[element['Prénom']].update({element['Année de naissance'] : element['Nombre']})
    else:
        index_rennais[element['Prénom']] = {element['Année de naissance'] : element['Nombre']}

print(index_rennais)

Pour __harmoniser les deux tables__, il faut également :

- passer en __majuscules__ les prénoms de la table rennaise.
- __convertir en entier les occurrences__ de naissance.
- rétablir une égalité à propos des prénoms peu donnés. La ville de Nantes ne répertoriant que les prénoms donnés au moins 5 fois dans l'année, il faut __exclure de la table rennaise les prénoms ayant une occurrence inférieure à 5__.

In [ ]:
index_rennais = {}

for element in naissances_rennaises:
    if int(element['Nombre']) >= 5:
        if element['Prénom'].upper() in index_rennais:
            index_rennais[element['Prénom'].upper()].update({element['Année de naissance'] : int(element['Nombre'])})
        else:
            index_rennais[element['Prénom'].upper()] = {element['Année de naissance'] : int(element['Nombre'])}

print(index_rennais)

> __Remarque :__ à ce stade, nous pourrions déjà comparer les occurrences de prénoms entre les deux villes, à l'aide des deux tables séparées. Voir ci-dessous.

In [ ]:
# Dans quelle ville a-t-on donné naissance au plus de ... en ...

prenom = input("Saisir un prénom : ")
annee = input("Saisir une année : ")

if prenom.upper() in index_nantais and prenom.upper() in index_rennais:
    if annee in index_nantais[prenom.upper()] and annee in index_rennais[prenom.upper()]:
        if index_nantais[prenom.upper()][annee] > index_rennais[prenom.upper()][annee]:
            print(f"C'est à Nantes qu'il est né le plus de {prenom} en {annee} : \
                  \n  - {index_nantais[prenom.upper()][annee]} {prenom} à Nantes\
                  \n  - {index_rennais[prenom.upper()][annee]} {prenom} à Rennes")
        elif index_nantais[prenom.upper()][annee] < index_rennais[prenom.upper()][annee]:
            print(f"C'est à Rennes qu'il est né le plus de {prenom} en {annee} : \
                  \n  - {index_rennais[prenom.upper()][annee]} {prenom} à Rennes\
                  \n  - {index_nantais[prenom.upper()][annee]} {prenom} à Nantes")
        else:
            print(f"Il est né autant de {prenom} à Nantes et Rennes en {annee} : \
            {index_nantais[prenom.upper()][annee]} naissances")
    else:
        print("Ce prénom n'a pas été donné cette année dans au moins une des villes")
else:
    print("Ce prénom n'a pas été donné dans au moins une des villes")

Nous avons utilisé les deux index séparément. Toutefois, il peut être plus cohérent d'__avoir l'ensemble de nos données dans une seule table__. Il faut pour cela faire une jointure.

Pour effectuer une __jointure entre nos deux index__, nous créerons un nouvel index basé sur la structure suivante :

```
index_naissances = {{'PRENOM': {'NANTES': {dates et occurrences nantaise}, 'RENNES': {dates et occurrences rennaise}}, ..........................................}
```
Ce qui permettra de faire des recherches rapides, sous la forme :
```
index_naissances['PRENOM']['VILLE']['ANNEE']
```

Le __descriteur commun aux deux tables, sur lequel sera basée la jointure__ est donc `PRENOM`.

In [ ]:
index_naissances = {}

for prenom_nantais, stats_nantaises in index_nantais.items():
    for prenom_rennais, stats_rennaises in index_rennais.items():
        if prenom_nantais == prenom_rennais:
            index_naissances[prenom_nantais] = {'NANTES': stats_nantaises,
                                                'RENNES': stats_rennaises}
print(index_naissances)

Ce nouvel index permet ainsi de faire des __recherches et des comparaisons rapides__, avec une bonne lisibilité du code.

Quelques exemples....

In [ ]:
index_naissances['GARANCE']

In [ ]:
index_naissances['GARANCE']['NANTES']

In [ ]:
index_naissances['GARANCE']['NANTES']['2018']

In [ ]:
index_naissances['GARANCE']['RENNES']['2018']

In [ ]:
index_naissances['GARANCE']['NANTES']['2018'] > index_naissances['GARANCE']['RENNES']['2018']

## Que retenir ?
### À minima...

- Lorsqu'on souhaite réunir sur une seule table des données présentes dans plusieurs tables, on doit faire une fusion.
- Il existe deux types de fusion de tables :
  - la réunion de tables si les descripteurs des deux tables sont strictement identiques.
  - la jointure de tables si un ou plusieurs descripteurs sont communs aux deux tables. On choisit alors le descripteur commun qui sera la base de la jointure.
- Savoir effectuer une réunion de tables.

### Au mieux...

- Savoir effectuer une jointure de tables (voir notebook P4).

---
[![Licence CC BY NC SA](https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png "licence Creative Commons CC BY-NC-SA")](http://creativecommons.org/licenses/by-nc-sa/3.0/fr/)
<p style="text-align: center;">Auteur : David Landry, Lycée Clemenceau - Nantes</p>